In [ ]:
import ray.tune as tune
from ray.rllib.agents.ppo import PPOTrainer
from ray.tune import register_env
from envs.env_creator import env_creator, ibgym_env_creator_rllib
from envs.IBGym_mod_envs import IBGymModded
from ppo.policy import LSTMPPOPolicy


## Configure Training

In [ ]:


register_env("IBGym-v1", ibgym_env_creator_rllib)

config = {
    # Environment (RLlib understands openAI gym registered strings).
    "env": "IBGym-v1",
    # Use 2 environment workers (aka "rollout workers") that parallelly
    # collect samples from their own environment clone(s).
    "num_workers": 12,
    "num_gpus": 1,

    # Change this to "framework: torch", if you are using PyTorch.
    # Also, use "framework: tf2" for tf2.x eager execution.
    "framework": "tf",
    "entropy_coeff": 0.0001,
    # "entropy_coeff_schedule":PiecewiseSchedule(endpoints=[(0, 0.01), (143000, 0.00001)]),
    "lr": 3e-5,
    "gamma": 0.994,
    "clip_param": 0.2,
    "seed": 5321,
    "num_sgd_iter": 2,
    "sgd_minibatch_size": 1000,

    # "vf_loss_coeff": 1e-9,
    # "vf_clip_param": 1e7,
    # Tweak the default model provided automatically by RLlib,
    # given the environment's observation- and action spaces.
    "model": {
        # == LSTM ==
        # Whether to wrap the model with an LSTM.
        "use_lstm": True,
        # Max seq len for training the LSTM, defaults to 20.
        "max_seq_len": 30,
        # Size of the LSTM cell.
        "lstm_cell_size": 64,
        # "use_attention": True,
        # "attention_num_transformer_units": 2,
        # "attention_dim": 128,
        # "vf_share_layers": True,
        # "fcnet_hiddens": [32, 32, 32],
        # "sgd_minibatch_size": 1024,
        "vf_share_layers": False,
        # Whether to feed a_{t-1} to LSTM (one-hot encoded if discrete).
        "lstm_use_prev_action": False,
        # Whether to feed r_{t-1} to LSTM.
        "lstm_use_prev_reward": False,
        # Whether the LSTM is time-major (TxBx..) or batch-major (BxTx..).
        "_time_major": False,
    },
    "train_batch_size": 32000,
    "timesteps_per_iteration": 32000,
    # "output": "tmp/ib-out",
    # Set up a separate evaluation worker set for the
    # `trainer.evaluate()` call after training (see below).
    "evaluation_num_workers": 3,
    # Only for evaluation runs, render the env.
    "evaluation_config": {
        "render_env": False,
    },
}

In [3]:
results = tune.run(
        PPOTrainer,
        config=config,
        name="industrial_benchmark",
        local_dir="tmp/ray_exp_logs",
        checkpoint_freq=5,
        # stop={"training_iteration": 5},
        sync_config=tune.SyncConfig(
            syncer=None  # Disable syncing
        )
    )

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_IBGym-v1_5b58d_00000,RUNNING,192.168.0.185:344251,13,206.826,831792,-230604,-212036,-263445,1000


Result for PPOTrainer_IBGym-v1_5b58d_00000:
  agent_timesteps_total: 895776
  counters:
    num_agent_steps_sampled: 895776
    num_agent_steps_trained: 895776
    num_env_steps_sampled: 895776
    num_env_steps_trained: 895776
  custom_metrics: {}
  date: 2022-10-14_12-13-56
  done: false
  episode_len_mean: 1000.0
  episode_media: {}
  episode_reward_max: -212035.63694766775
  episode_reward_mean: -230745.94534207933
  episode_reward_min: -303472.80645493016
  episodes_this_iter: 60
  episodes_total: 888
  experiment_id: 60b1151648774b57ab89dde1fea079b7
  hostname: hamza-Legion-5-15ACH6H
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 2.9999999242136255e-05
          entropy: 3.1898248195648193
          entropy_coeff: 9.999999747378752e-05
          kl: 0.0020165974274277687
          model: {}
          policy_loss: -0.0016078799962997437
          total_loss: 9.998476028442383
          vf_explained_va

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_IBGym-v1_5b58d_00000,RUNNING,192.168.0.185:344251,14,222.333,895776,-230746,-212036,-303473,1000


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_IBGym-v1_5b58d_00000,RUNNING,192.168.0.185:344251,14,222.333,895776,-230746,-212036,-303473,1000


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_IBGym-v1_5b58d_00000,RUNNING,192.168.0.185:344251,14,222.333,895776,-230746,-212036,-303473,1000


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_IBGym-v1_5b58d_00000,RUNNING,192.168.0.185:344251,147,2300.72,9.40565e+06,-195572,-188126,-233796,1000


2022-10-14 12:48:55,070	WARNING tune.py:682 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_IBGym-v1_5b58d_00000,RUNNING,192.168.0.185:344251,147,2300.72,9.40565e+06,-195572,-188126,-233796,1000


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_IBGym-v1_5b58d_00000,RUNNING,192.168.0.185:344251,147,2300.72,9.40565e+06,-195572,-188126,-233796,1000


2022-10-14 12:48:57,155	ERROR tune.py:743 -- Trials did not complete: [PPOTrainer_IBGym-v1_5b58d_00000]
2022-10-14 12:48:57,156	INFO tune.py:747 -- Total run time: 2339.42 seconds (2337.31 seconds for the tuning loop).
2022-10-14 12:48:57,156	WARNING tune.py:753 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`


In [6]:



last_checkpoint = results.get_last_checkpoint()._local_path
config["num_workers"] = 1

lstm_ppo_policy = LSTMPPOPolicy(config=config,checkpoint_path=last_checkpoint)
save_path = 'ppo/lstm_ppo_policy.pkl'
lstm_ppo_policy.save(save_path)

In [7]:
results.get_last_checkpoint()._local_path

'/home/hamza/PycharmProjects/StateCompression/tmp/ray_exp_logs/industrial_benchmark/PPOTrainer_IBGym-v1_5b58d_00000_0_2022-10-14_12-09-57/checkpoint_000145/checkpoint-145'